In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! git clone https://github.com/fra31/auto-attack.git

Cloning into 'auto-attack'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 672 (delta 162), reused 159 (delta 141), pack-reused 463
Receiving objects: 100% (672/672), 40.63 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [4]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-uhmj1ga0
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-uhmj1ga0
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=95df478d6257e6a6487a9ee8de44e5b85ca4ca6201fd8139d20f1eee7b36b596
  Stored in directory: /tmp/pip-ephem-wheel-cache-jqm1872k/wheels/b7/17/95/4b16fafe1b84fdabd247eb5f01b41165bd98d4c82d64ab93d2
Successfully built autoattack


In [5]:
# Copy Files
!cp /content/drive/MyDrive/ML-Project/Test4.rar /content

In [6]:
! unrar x /content/Test4.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/Test4.rar

Extracting  ckpt.pth                                                       5% 10% 15% 20% 26% 31% 36% 41% 47% 52% 57% 62% 67% 73% 78% 83% 88% 94% 99%  OK 
Extracting  resnet.py                                                     99%  OK 
All OK


In [7]:
!cp -r /content/resnet.py /content/auto-attack/autoattack/examples

In [8]:
%%shell
cat > /content/auto-attack/autoattack/examples/eval.py << EOF

import os
import argparse
from pathlib import Path
import warnings

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

import sys
sys.path.insert(0,'..')
from resnet import ResNet34

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='./data')
    parser.add_argument('--norm', type=str, default='L2')
    parser.add_argument('--epsilon', type=float, default=0.3)
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--model', type=str, default='/content/ckpt.pth')
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--n_ex', type=int, default=1000)
    parser.add_argument('--individual', action='store_true')
    parser.add_argument('--save_dir', type=str, default='./results')
    parser.add_argument('--batch_size', type=int, default=500)
    parser.add_argument('--log_path', type=str, default='./log_file.txt')
    parser.add_argument('--version', type=str, default='standard')
    parser.add_argument('--state-path', type=Path, default=None)

    args = parser.parse_args()

    # load model
    model = ResNet34()
    #//////////////////////////////////////////////
    model = nn.DataParallel(model)
    loaded = torch.load(args.model)
    model.load_state_dict(loaded['net'])
    #//////////////////////////////////////////////
    model.cuda()
    model.eval()

    # load data
    transform_list = [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    transform_chain = transforms.Compose(transform_list)
    item = datasets.CIFAR10(root=args.data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

    # create save dir
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load attack
    from autoattack import AutoAttack
    adversary = AutoAttack(model, norm=args.norm, eps=args.epsilon, log_path=args.log_path,
        version=args.version)

    l = [x for (x, y) in test_loader]
    x_test = torch.cat(l, 0)
    l = [y for (x, y) in test_loader]
    y_test = torch.cat(l, 0)

    # example of custom version
    if args.version == 'custom':
        adversary.attacks_to_run = ['apgd-ce', 'fab']
        adversary.apgd.n_restarts = 2
        adversary.fab.n_restarts = 2

    # run attack and save images
    with torch.no_grad():
        if not args.individual:
            adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
                bs=args.batch_size, state_path=args.state_path)

            torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
                args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

        else:
            # individual version, each attack is run on all test points
            adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
                y_test[:args.n_ex], bs=args.batch_size)

            torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_{}_cheap_{}.pth'.format(
                args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))

/bin/bash: line 86: warning: here-document at line 0 delimited by end-of-file (wanted `EOF')


#AA-L2

##eps = 0.3

In [9]:
epsilon = 0.3
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

100% 170498071/170498071 [00:13<00:00, 12790179.92it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 194 out of 500 successfully perturbed
apgd-ce - 2/2 - 177 out of 454 successfully perturbed
robust accuracy after APGD-CE: 58.30% (total time 94.4 s)
apgd-t - 1/2 - 3 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 83 successfully perturbed
robust accuracy after APGD-T: 58.00% (total time 618.3 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 80 successfully perturbed
robust accuracy after FAB-T: 58.00% (total time 1577.6 s)
square - 1/2 - 6 out of 500 successfully perturbed
square - 2/2 - 0 out of 80 successfully perturbed
robust accuracy after SQUARE: 57.40% (total time 2924.1 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 57.40%
Finish!


##eps = 0.4

In [10]:
epsilon = 0.4
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 194 out of 500 successfully perturbed
apgd-ce - 2/2 - 177 out of 454 successfully perturbed
robust accuracy after APGD-CE: 58.30% (total time 95.0 s)
apgd-t - 1/2 - 6 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 83 successfully perturbed
robust accuracy after APGD-T: 57.70% (total time 615.1 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 77 successfully perturbed
robust accuracy after FAB-T: 57.70% (total time 1567.8 s)
square - 1/2 - 5 out of 500 successfully perturbed
square - 2/2 - 0 out of 77 successfully perturbed
robust accuracy after SQUARE: 57.20% (total time 2910.9 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 57.20%
Finish!


##eps = 0.5

In [11]:
epsilon = 0.5
norm = 'L2'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 195 out of 500 successfully perturbed
apgd-ce - 2/2 - 180 out of 454 successfully perturbed
robust accuracy after APGD-CE: 57.90% (total time 95.0 s)
apgd-t - 1/2 - 4 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 79 successfully perturbed
robust accuracy after APGD-T: 57.50% (total time 611.4 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 75 successfully perturbed
robust accuracy after FAB-T: 57.50% (total time 1563.2 s)
square - 1/2 - 7 out of 500 successfully perturbed
square - 2/2 - 0 out of 75 successfully perturbed
robust accuracy after SQUARE: 56.80% (total time 2904.8 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 56.80%
Finish!


#AA-Linf

##eps = 4/255

In [12]:
epsilon = 4/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 304 out of 500 successfully perturbed
apgd-ce - 2/2 - 271 out of 454 successfully perturbed
robust accuracy after APGD-CE: 37.90% (total time 95.0 s)
apgd-t - 1/1 - 4 out of 379 successfully perturbed
robust accuracy after APGD-T: 37.50% (total time 431.7 s)
fab-t - 1/1 - 0 out of 375 successfully perturbed
robust accuracy after FAB-T: 37.50% (total time 1058.1 s)
square - 1/1 - 0 out of 375 successfully perturbed
robust accuracy after SQUARE: 37.50% (total time 1919.4 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 37.50%
Finish!


##eps = 6/255

In [13]:
epsilon = 6/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 356 out of 500 successfully perturbed
apgd-ce - 2/2 - 313 out of 454 successfully perturbed
robust accuracy after APGD-CE: 28.50% (total time 95.0 s)
apgd-t - 1/1 - 10 out of 285 successfully perturbed
robust accuracy after APGD-T: 27.50% (total time 346.6 s)
fab-t - 1/1 - 0 out of 275 successfully perturbed
robust accuracy after FAB-T: 27.50% (total time 812.0 s)
square - 1/1 - 0 out of 275 successfully perturbed
robust accuracy after SQUARE: 27.50% (total time 1448.5 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 27.50%
Finish!


##eps = 8/255

In [14]:
epsilon = 8/255
norm = 'Linf'
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm
print("Finish!")

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 95.40%
apgd-ce - 1/2 - 396 out of 500 successfully perturbed
apgd-ce - 2/2 - 362 out of 454 successfully perturbed
robust accuracy after APGD-CE: 19.60% (total time 94.9 s)
apgd-t - 1/1 - 9 out of 196 successfully perturbed
robust accuracy after APGD-T: 18.70% (total time 264.6 s)
fab-t - 1/1 - 0 out of 187 successfully perturbed
robust accuracy after FAB-T: 18.70% (total time 580.5 s)
square - 1/1 - 0 out of 187 successfully perturbed
robust accuracy after SQUARE: 18.70% (total time 1012.3 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 18.70%
Finish!
